### https://www.kaggle.com/vincentman0403/dcgan-on-mnist 


In [1]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
import time
import os
from PIL import Image

Using TensorFlow backend.


In [2]:
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)
latent_dim = 100
root = 'MNIST_DCGAN_results/'
model = 'MNIST_DCGAN_'

# plotting data structure
train_hist = {}
train_hist['D_losses'] = []
train_hist['G_losses'] = []
train_hist['per_epoch_ptimes'] = []
train_hist['total_ptime'] = []

In [3]:
def build_generator():
    model = Sequential()
    model.add(Dense(128 * 7 * 7, activation="relu", input_dim=latent_dim))
    model.add(Reshape((7, 7, 128)))
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(Conv2D(channels, kernel_size=3, padding="same"))
    model.add(Activation("tanh"))
    model.summary()
    noise = Input(shape=(latent_dim,))
    img = model(noise)
    return Model(noise, img)

## Define a function to build a discriminator

In [4]:
def build_discriminator(pick_model=0):
    if pick_model == 0:
        model = Sequential()
        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))
        model.summary()
    
    img = Input(shape=img_shape)
    validity = model(img)
    return Model(img, validity)

## Build GAN

In [5]:
optimizer = Adam(0.0002, 0.5)

# build discriminator
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy'])

# build generator
generator = build_generator()
z = Input(shape=(100,))
img = generator(z)

# For the combined model we will only train the generator
discriminator.trainable = False

# The discriminator takes generated images as input and determines validity
valid = discriminator(img)

# The combined model  (stacked generator and discriminator)
# Trains the generator to fool the discriminator
combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_1

## Define a function to train GAN

In [6]:
def train(epochs, batch_size=128, save_interval=50, ratio = (1,1)): ## ratio G:D
    os.makedirs('images', exist_ok=True)
    
    # Load the dataset
    (X_train, _), (_, _) = mnist.load_data()

    # Rescale -1 to 1
    X_train = X_train / 127.5 - 1.
    X_train = np.expand_dims(X_train, axis=3)

    # Adversarial ground truths
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    
    
    for epoch in range(epochs):
        # Select a random real images
        epoch_start_time = time.time()
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        real_imgs = X_train[idx]

        # Sample noise and generate a batch of fake images
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        fake_imgs = generator.predict(noise)
        
      #implement ratio of D  
        if epoch % ratio[0] == 0:
            # Train the discriminator
            D_loss_real = discriminator.train_on_batch(real_imgs, valid)
            D_loss_fake = discriminator.train_on_batch(fake_imgs, fake)
            D_loss = 0.5 * np.add(D_loss_real, D_loss_fake)
      #implement ratio of G
        if epoch % ratio[1] == 0:
            # Train the generator
            g_loss = combined.train_on_batch(noise, valid)
            
        # get epoch timing
        epoch_end_time = time.time()
        per_epoch_ptime = epoch_end_time - epoch_start_time
            
        # save data for plotting
        train_hist['D_losses'].append(D_loss[0])
        train_hist['G_losses'].append(g_loss)
        train_hist['per_epoch_ptimes'].append(per_epoch_ptime)    
        
        # If at save interval
        if epoch % save_interval == 0:
            # Print the progress
            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f] [epoch time: %.2f]" % (epoch, D_loss[0], 100 * D_loss[1], g_loss, per_epoch_ptime))
            # Save generated image samples
            save_imgs(epoch)

In [7]:
def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
            axs[i, j].axis('off')
            cnt += 1
    fig.savefig("images/mnist_%d.png" % epoch)
    plt.close()

In [9]:
def show_train_hist(hist, show = False, save = False, path = 'Train_hist.png'):
    x = range(len(hist['D_losses']))

    y1 = hist['D_losses']
    y2 = hist['G_losses']

    plt.plot(x, y1, label='D_loss')
    plt.plot(x, y2, label='G_loss')

    plt.xlabel('Epoch')
    plt.ylabel('Loss')

    plt.legend(loc=4)
    plt.grid(True)
    plt.tight_layout()

    if save:
        plt.savefig(path)

    if show:
        plt.show()
    else:
        plt.close()

## Train GAN

In [8]:
start = time.time()

# train_hist = {}
#     train_hist['D_losses'] = []
#     train_hist['G_losses'] = []
#     train_hist['per_epoch_ptimes'] = []
#     train_hist['total_ptime'] = []

train(epochs=100, batch_size=32, save_interval=1, ratio=(1,1)) ## ratio G:D

end = time.time()
elapsed_train_time = 'elapsed training time: {} min, {} sec '.format(int((end - start) / 60),
                                                                     int((end - start) % 60))
train_hist['total_ptime'].append(elapsed_train_time)

print(elapsed_train_time)

show_train_hist(train_hist, save=True, path=root + model + 'train_hist.png')

Instructions for updating:
Use tf.cast instead.


C:\Users\Jack\venv\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 1.208578, acc.: 29.69%] [G loss: 0.662138] [epoch time: 13.01]
1 [D loss: 0.637409, acc.: 64.06%] [G loss: 0.778305] [epoch time: 0.14]
2 [D loss: 0.635397, acc.: 64.06%] [G loss: 1.101668] [epoch time: 0.14]
3 [D loss: 0.450472, acc.: 78.12%] [G loss: 1.132891] [epoch time: 0.14]
4 [D loss: 0.258223, acc.: 95.31%] [G loss: 1.218686] [epoch time: 0.14]
5 [D loss: 0.219270, acc.: 96.88%] [G loss: 1.011388] [epoch time: 0.14]
6 [D loss: 0.243134, acc.: 96.88%] [G loss: 1.058210] [epoch time: 0.14]
7 [D loss: 0.115336, acc.: 100.00%] [G loss: 0.523877] [epoch time: 0.14]
8 [D loss: 0.211301, acc.: 93.75%] [G loss: 0.888975] [epoch time: 0.14]
9 [D loss: 0.354175, acc.: 81.25%] [G loss: 1.029729] [epoch time: 0.14]
10 [D loss: 0.789673, acc.: 59.38%] [G loss: 1.687781] [epoch time: 0.14]
11 [D loss: 0.740547, acc.: 57.81%] [G loss: 2.329051] [epoch time: 0.14]
12 [D loss: 0.952504, acc.: 51.56%] [G loss: 1.792017] [epoch time: 0.14]
13 [D loss: 0.752575, acc.: 57.81%] [G loss: 1

NameError: name 'show_train_hist' is not defined

In [ ]:
os.makedirs('saved_model_weights', exist_ok=True)
generator.save_weights('saved_model_weights/generator_weights.h5')
discriminator.save_weights('saved_model_weights/discriminator_weights.h5')
combined.save_weights('saved_model_weights/combined_weights.h5')

## Show generated MNIST images

In [ ]:
Image.open('images/mnist_1000.png')

In [ ]:
Image.open('images/mnist_9000.png')

## Reference
[Keras - DCGAN](https://github.com/eriklindernoren/Keras-GAN#dcgan)